In [1]:
# import of adj_mat
# import of nodes_cluster
# import of news_data
# import of crypto_data

# Import section

## adj_mat

## nodes_cluster

## news_data

## crypto_data

In [5]:
import yfinance as yf
import os
import pandas as pd
import numpy as np
from yahoofinancials import YahooFinancials
#input value, data from yahoo finance
value = input("Please enter the currency: \n")
ticker = yf.Ticker(f'{value}-USD')
df_ta = ticker.history(period="5y")
df_ta.drop(['Open', 'High', 'Low', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
#add index value
p = df_ta.index.values
df_ta.insert( 0, column="Date",value = p)
#Calculate RSI
delta = df_ta['Close'].diff(1) #difference from the previous day
delta = delta.dropna()
#get the positive gains(up) and negative gains(down)
up = delta.copy()
down = delta.copy()
up[up < 0] = 0
down[down > 0] = 0
#Get the time period 14 days
period = 14
#calculate the average gain and loss
AVG_Gain = up.rolling(window=period).mean()
AVG_Loss = abs(down.rolling(window=period).mean())
#Calculate RSI
RS = AVG_Gain / AVG_Loss #relative strength
#RSI
RSI = 100.0 - (100.0/(1.0 + RS))
#Put it all together
new_df = pd.DataFrame()
new_df['Close'] = df_ta['Close']
new_df['RSI'] = RSI
new_df
#Get Buy and Sell Signals
conditions = [
    (new_df['RSI'] < 80) & (new_df['RSI'] > 20),
    (new_df['RSI'] < 80),
    (new_df['RSI'] > 20)]
choices = [0, -2, 2]
df_ta['signal_rsi'] = np.select(conditions, choices, default='0')
#Bollinger Bands indicator
period= 20
#Simple Moving Average
df_ta['SMA'] = df_ta['Close'].rolling(window=period).mean()
#Standard deviation
df_ta['STD'] = df_ta['Close'].rolling(window=period).std()
#upper band
df_ta['Upper'] = df_ta['SMA'] + (df_ta['STD']*2)
#lower band
df_ta['Lower'] = df_ta['SMA'] - (df_ta['STD']*2)
#Create Signal Value
conditions = [
    (df_ta.Lower > df_ta.Close) & (df_ta.Upper < df_ta.Close),
    (df_ta.Lower > df_ta.Close),
    (df_ta.Upper < df_ta.Close)]
choices = [0, 2, -2]
df_ta['signal_bb'] = np.select(conditions, choices, default='0')
df_ta = df_ta.dropna()
#On Balance Volume indicator
df_ta['Daily Returns'] = df_ta['Close'].pct_change()
df_ta['signal_obv'] = 0
df_ta.loc[df_ta['Daily Returns'] > 0, 'signal_obv'] = 2
df_ta.loc[df_ta['Daily Returns'] < 0, 'signal_obv'] = -2
df_ta.dropna(inplace= True)
#MVA indicator
#Create the simple MVA
def SMA(data, period = 30, column ='Close'):
    return data[column].rolling(window=period).mean()
#Creating the Exponential MVA
def EMA(data, period=20, column='Close'):
    return data[column].ewm(span=period, adjust=False).mean()
#Calculate MVA Convergence/Divergence
def MACD(data, period_long=28, period_short=12, period_signal=9, column='Close'):
    #calculate the short term exponential MVA
    ShortEMA = EMA(data, period_short, column=column)
    #calculate the long term exponential MVA
    LongEMA = EMA(data, period_long, column=column)
    #calculate the MVA Convergence/Divergence MACD
    data['MACDA'] = ShortEMA - LongEMA
    #calculate the signal line
    data['Signal_Line'] = EMA(data, period_signal, column='MACD')
    return data
df_ta['SMA'] = SMA(df_ta)
df_ta['EMA'] = EMA(df_ta)
#Create a Simple Moving Average indicator
def SMA(data, period = 30, column = 'Close'):
    return data[column].rolling(window=period).mean()
#Create two new columns to store 20 and 50 day SMA
df_ta['SMA20']=SMA(df_ta, 20)
df_ta['SMA50']=SMA(df_ta, 50)
#Get Buy and Sell Signals
df_ta['signal_mva'] = np.where(df_ta['SMA20']> df_ta['SMA50'], 2, -2)
# Delete redundant data
df_ta.drop(['SMA', 'STD', 'Upper', 'Lower','Daily Returns', 'EMA','SMA20','SMA50'], axis=1, inplace=True)
#prepare datatype
df_ta['signal_rsi'] = df_ta['signal_rsi'].astype('int')
df_ta['signal_bb'] = df_ta['signal_bb'].astype('int')
df_ta['signal_mva'] = df_ta['signal_mva'].astype('int')
df_ta['signal_obv'] = df_ta['signal_obv'].astype('int')
#add calculated average
zaehler =  df_ta.apply(lambda x: (0.3 * x['signal_bb']) + (0.3 * x['signal_mva']) + (0.2 * x['signal_obv']) +(0.2 * x['signal_rsi']), axis=1)
df_ta['calculated_average'] = zaehler / 1
#Plot Frame
df_ta

Please enter the currency: 
BTC


,Date,Close,Volume,signal_rsi,signal_bb,signal_obv,signal_mva,calculated_average
Date,,,,,,,,
2017-02-06,2017-02-06,1038.150024,111762000,0,0,2,-2,-0.2
2017-02-07,2017-02-07,1061.349976,146007008,2,0,2,-2,0.2
2017-02-08,2017-02-08,1063.069946,201855008,2,0,2,-2,0.2
2017-02-09,2017-02-09,994.382996,407220000,0,0,-2,-2,-1.0
2017-02-10,2017-02-10,988.674011,190452000,0,0,-2,-2,-1.0
...,...,...,...,...,...,...,...,...
2022-01-13,2022-01-13,42591.570312,47691135082,0,0,-2,-2,-1.0
2022-01-14,2022-01-14,43099.699219,23577403399,0,0,2,-2,-0.2
2022-01-15,2022-01-15,43177.398438,18371348298,0,0,2,-2,-0.2


# Merging section (probably needed)

# Model definition (probably needed)

## max. cluster strength

In [2]:
# cluster strength

## input:
### date
### cluster annotation per node, per day
## throughput1:
### number of neighbors inside cluster per node, per day
### number of total neighbors per node, per day (degree per node, per day)
## throughput2:
### (number of neighbors inside cluster per node, per day) / (degree per node, per day)
## output:
### lowest value of throughput2, per day

For the scores it is important to note that the lowest value indicates that there is no stimulation for a shock reaction.
In the model n = -1 means that there is the lowest possible amount of stimulation from the daily news sentiment whereas n = 1 means that there is the highest possible amount. 

## sentiment score

## indicator score

## model payoffs

The mathematical basis for the cascade model provides a logistic function (I). In addition to the current date, news sentiment and indicator scores are summed up and serve as model input. Based on (I) the model calculates the perceived payoffs for strategy A and strategy B. These payoff values function as global parameters for the whole network, implying that every node inside the network perceives the respective payoffs in the same way. 


A crucial design decision that has been made is that the payoff for strategy B is the counter-calue of the payoff for strategy B. This decision should emphasize that strategy B is simply the absence of strategy A. Therefore, the inequality (II), which decides whether or not a complete strategy A cascade is possible, is reduced to (II.1). In order for a complete strategy A cascade to be hence possible, inside the network on a specific date, there cannot exist a cluster with a cluster strength p greater than a.

- I: equality for payoff a
- II: inequality for complete cascade p > 1 - b
- II.1: p > a

In [39]:
# model_payoffs:

## input:
### x --> linear combination of scores
### date
## output:
### payoff a --> a = 1/(1+exp(-x)), per day
### payoff b --> b = 1 - a, per day

# strategy A --> shock reaction (hype/panic)
# strategy B --> indifferent reaction

# threshold:

## input:
### payoff for strategy a
### payoff for strategy b
## output:
### threshold value q --> q = b / (a + b)


max cluster strength p has to be greater than a (p>a) for strategy A to cause a complete cascade

- this is because b = 1 - a and q = b/(a+b) (--> a+b = 1 --> q = b/1)

- original threshold formula: p > 1 - q --> p > 1 - b --> p > a (--> b=1-a --> a=1-b)

- so in order for a cluster to break a complete strategy A cascade it has to have a strength greater than a

In [38]:
import math


d = ['november']
n = -1
t = -1

def cascade_model(date, news, ta):
    
    #sum of scores
    x = news + ta
    
    #payoffs
    a = 1 / (1 + math.exp(-x))
    b = 1 - a
    
    #threshold
    q = b / (a + b)
    
    #output
    model_output_dict = {}
    
    for d in date:
        model_output_dict[d] = [(a,b), 1 - q]
                                
    return model_output_dict

cascade_model(date=d, news=n, ta=t)

{'november': [(0.11920292202211755, 0.8807970779778824), 0.11920292202211757]}